In [ ]:
!wget https://hf.co/danjacobellis/walloc/resolve/main/Stereo_Li_48c_J8_nf8_v1.0.2.pth

In [ ]:
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs

In [1]:
import torch
import numpy as np
import torchaudio
import tempfile
from cdpam import CDPAM
from demucs.separate import Separator
from IPython.display import Audio
from datasets import load_dataset, Dataset
from walloc import walloc
from spauq.core.metrics import spauq_eval
from fastprogress.fastprogress import progress_bar
class Config: pass

In [2]:
device = "cuda"
checkpoint = torch.load("Stereo_Li_48c_J8_nf8_v1.0.2.pth",map_location="cpu",weights_only=False)
codec_config = checkpoint['config']
codec = walloc.Codec1D(
    channels = codec_config.channels,
    J = codec_config.J,
    Ne = codec_config.Ne,
    Nd = codec_config.Nd,
    latent_dim = codec_config.latent_dim,
    latent_bits = codec_config.latent_bits,
    lightweight_encode = codec_config.lightweight_encode,
    post_filter = codec_config.post_filter
).to(device)
codec.load_state_dict(checkpoint['model_state_dict'])
codec.eval();

/home/dgj335/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [3]:
device = "cuda"
separator = Separator()
cdpam_loss = CDPAM()
MUSDB = load_dataset("danjacobellis/musdb18HQ", split='validation')

/home/dgj335/.local/lib/python3.10/site-packages/cdpam/cdpam.py:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(modfolder,map_location="cpu")['state']


Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

In [4]:
separator = Separator()

In [5]:
def pad(audio, p=2**16):
    B,C,L = audio.shape
    padding_size = (p - (L % p)) % p
    if padding_size > 0:
        audio = torch.nn.functional.pad(audio, (0, padding_size), mode='constant', value=0)
    return audio

In [6]:
max_duration=44100*180
SSDR = []
SRDR = []
PSNR = []
CDPAM_dB = []
res_red = torchaudio.transforms.Resample(44100, 16536).to(device)
res_inc = torchaudio.transforms.Resample(16536, 44100).to(device)
with torch.no_grad():
    for i_sample in progress_bar(range(0, len(MUSDB), 5)):
        SSDR.append([])
        SRDR.append([])
        PSNR.append([])
        CDPAM_dB.append([])
        y = []
        for i_instr in range(5):
            sample = MUSDB[i_sample+i_instr]
            instr = sample['instrument']
            x, fs = torchaudio.load(sample['audio']['bytes'])
            x = x[:,:max_duration]
            L = x.shape[-1]
            x_padded = pad(x.unsqueeze(0), 2**16).to(device)

            X = codec.wavelet_analysis(x_padded,codec.J)
            Y = codec.encoder(X)
            X_hat = codec.decoder(Y)
            x_hat = codec.wavelet_synthesis(X_hat,codec.J)
            x_hat = codec.post(x_hat)
                              
            if i_instr == 0:
                pred = separator.separate_tensor(x_hat[0])
                
            else:
                ℓ = min(x_padded.shape[-1], pred[1][instr].shape[-1])
                SDR = spauq_eval(
                    reference=x_padded[0,:,:ℓ].to("cpu"),
                    estimate=pred[1][instr][:,:ℓ].to("cpu"),
                    fs = fs
                )
                psnr = 20*np.log10(2) - 10*np.log10(torch.nn.functional.mse_loss(
                    x_padded[0,:,:ℓ],
                    pred[1][instr][:,:ℓ]
                ).item())
                cdpam = cdpam_loss.forward(
                    wav_in=x_padded[0,:,:ℓ],
                    wav_out=pred[1][instr][:,:ℓ]
                )
                SSDR[-1].append(SDR['SSR'])
                SRDR[-1].append(SDR['SRR'])
                PSNR[-1].append(psnr)
                CDPAM_dB[-1].append(-np.log10(cdpam.mean().item()))
SSDR = torch.tensor(SSDR)
SRDR = torch.tensor(SRDR)
PSNR = torch.tensor(PSNR)
CDPAM_dB = torch.tensor(CDPAM_dB)

/home/dgj335/.local/lib/python3.10/site-packages/spauq/core/preprocessing.py:325: UserWarning: No forgive_mode specified, defaulting to `none`
  warnings.warn(


In [7]:
metrics = {
    'SSDR': SSDR,
    'SRDR': SRDR,
    'PSNR': PSNR,
    'CDPAM_dB': CDPAM_dB,
}

instruments = ['other', 'drums', 'bass', 'vocals']

data = {}
for metric_name, metric_tensor in metrics.items():
    for i, instr in enumerate(instruments):
        column_name = f"{metric_name}_{instr}"
        data[column_name] = metric_tensor[:, i].tolist()
dataset = Dataset.from_dict(data)

In [8]:
dataset.push_to_hub("danjacobellis/LSDIR_demucs_10x_walloc",split='validation')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/danjacobellis/LSDIR_demucs_10x_walloc/commit/2f7ad79df0a89327444d2dc160edab13f689f6b6', commit_message='Upload dataset', commit_description='', oid='2f7ad79df0a89327444d2dc160edab13f689f6b6', pr_url=None, pr_revision=None, pr_num=None)

In [9]:
dataset.to_pandas().mean(axis=0)

SSDR_other         13.210964
SSDR_drums         15.545982
SSDR_bass          15.413898
SSDR_vocals         8.713092
SRDR_other          2.591630
SRDR_drums          5.371202
SRDR_bass          -0.345901
SRDR_vocals        -4.613902
PSNR_other         34.775065
PSNR_drums         37.343891
PSNR_bass          39.475253
PSNR_vocals        38.148983
CDPAM_dB_other      4.582460
CDPAM_dB_drums      4.108161
CDPAM_dB_bass       4.833813
CDPAM_dB_vocals     4.511948
dtype: float64

In [10]:
dataset.to_pandas()

,SSDR_other,SSDR_drums,SSDR_bass,SSDR_vocals,SRDR_other,SRDR_drums,SRDR_bass,SRDR_vocals,PSNR_other,PSNR_drums,PSNR_bass,PSNR_vocals,CDPAM_dB_other,CDPAM_dB_drums,CDPAM_dB_bass,CDPAM_dB_vocals
0,10.963948,25.382663,21.176526,1.672758,1.116825,11.005268,9.871680,-2.436919,34.765188,39.405682,40.143610,35.867936,3.140020,3.389798,4.663171,3.836347
1,13.711838,12.211710,21.039412,15.655858,4.395452,3.846703,10.252628,6.734602,33.207471,36.542568,42.893640,37.053958,4.665171,3.622383,4.558417,4.296459
2,18.728068,8.492473,11.019325,5.765542,6.252711,1.989378,4.709165,-1.378388,33.404194,38.521333,37.917358,36.161138,5.005614,4.139701,4.580261,4.507821
3,8.397912,18.595190,12.661220,0.000000,1.198590,8.219370,5.255046,-36.182570,32.817107,36.468019,34.769070,38.523777,3.978433,4.250344,4.742420,5.132533
4,15.986800,0.009492,12.674804,7.667632,4.724560,-30.867879,6.061482,-2.725874,37.745976,43.711763,39.759666,45.738639,5.060734,4.869245,4.539175,5.018162
5,12.910101,24.002300,27.442047,0.000000,3.803190,11.685765,15.723302,-80.000000,38.506711,40.668442,40.364430,66.725722,4.310414,4.381319,5.162961,7.595604
6,15.884034,22.167421,9.298949,8.013702,4.328444,9.344784,4.067973,2.493846,33.601908,40.304852,42.767786,35.085225,4.875728,4.373563,5.292140,4.214181
7,25.261382,11.095688,21.730447,0.143696,7.786348,4.413152,10.637873,-20.331785,33.752878,42.579946,40.501705,35.088243,5.018775,4.453663,4.165101,4.241839
8,9.165112,7.809579,21.008229,11.037278,0.212788,1.634696,11.118904,3.936208,34.240273,36.567865,40.082463,33.738446,4.243302,3.961566,5.245270,4.319586
9,16.251555,18.733947,6.713804,6.706180,5.383060,8.402164,1.667671,1.819137,33.345279,38.940068,38.334983,35.682755,5.057449,4.062787,4.667856,4.337536
